In [38]:
import numpy
import bokeh.plotting as bplot
bplot.output_notebook()
from matplotlib import pyplot
%matplotlib notebook

BokehJS successfully loaded.

In [39]:
from soapy import WFS, confParse
from aotools import circle

In [40]:
SOAPY_CONF = "conf/test_conf.py"

In [41]:
# load Soapy config
config = confParse.Configurator(SOAPY_CONF)
config.loadSimParams()
config.wfss[0].pxlsPerSubap = 30
mask = circle.circle(config.sim.pupilSize/2., config.sim.simSize)

In [42]:
# One arcec in tilt Amp across scope
tilt_rad = 1./3600 * numpy.pi/180
tilt_amp = tilt_rad*config.tel.telDiam/2.
tilt_amp/(550e-9)

35.25917680796625

In [56]:
PXL_SCALE = 0.4 #asec/pxl

In [57]:
# Measure current pixel scale
maxTilt = 4*PXL_SCALE # arcsecs
tilts_asec = numpy.linspace(-maxTilt, maxTilt, 10)

tilts_rad = (tilts_asec/3600.) * (numpy.pi/180)
# Must convert arcsecs of tilt to amp in nm
tiltAmps = 1e9 * (tilts_rad * config.tel.telDiam/2.)# in nm amp

# Make a unit tilt to multiply
coords = numpy.linspace(-1, 1, config.sim.pupilSize)
coords = numpy.pad(coords, pad_width=[config.sim.simPad,]*2, mode="constant")
tilt = numpy.meshgrid(coords, coords)[0]

measuredTilts = numpy.zeros((tiltAmps.shape[0]))
for i, A in enumerate(tiltAmps):
    config.wfss[0].subapFOV = config.wfss[0].pxlsPerSubap*PXL_SCALE
    wfs = WFS.ShackHartmann(config.sim, config.wfss[0], config.atmos, config.lgss[0], mask)
    
    aTilt = tilt*A
    slopes = wfs.frame(aTilt)
    measuredTilts[i] = -1*slopes[wfs.activeSubaps:].mean()
    
    # Check tilt is what I think it is
    dAmp = (aTilt.max() - aTilt.min())*1e-9
    angle = (dAmp/config.tel.telDiam) * (180./numpy.pi) * 3600

In [58]:
A = numpy.vstack([tilts_asec, numpy.ones(len(tilts_asec))]).T
pxlScale, bias = numpy.linalg.lstsq(A, measuredTilts)[0]
print("Measured Pixel Scale: {} pxls/arcsec".format(1./pxlScale))

Measured Pixel Scale: 0.3794981031 pxls/arcsec


In [59]:
plt = bplot.figure()
plt.line(tilts_asec, measuredTilts)
plt.line(tilts_asec, pxlScale*tilts_asec + bias, color="black")
bplot.show(plt)

In [47]:
def getMeasurePixelScale(maxTilt, config, samples=50):
    
    # Measure current pixel scale
    tilts_asec = numpy.linspace(-maxTilt, maxTilt, samples)

    tilts_rad = (tilts_asec/3600.) * (numpy.pi/180)
    # Must convert arcsecs of tilt to amp in nm
    tiltAmps = 1e9 * (tilts_rad * config.tel.telDiam/2.)# in nm amp

    # Make a unit tilt to multiply
    coords = numpy.linspace(-1, 1, config.sim.pupilSize)
    coords = numpy.pad(coords, pad_width=[config.sim.simPad,]*2, mode="constant")
    tilt = numpy.meshgrid(coords, coords)[0]

    measuredTilts = numpy.zeros((tiltAmps.shape[0]))
    for i, A in enumerate(tiltAmps):
        wfs = WFS.ShackHartmann(config.sim, config.wfss[0], config.atmos, config.lgss[0], mask)

        aTilt = tilt*A
        slopes = wfs.frame(aTilt)
        measuredTilts[i] = -1*slopes[wfs.activeSubaps:].mean()

        # Check tilt is what I think it is
        dAmp = (aTilt.max() - aTilt.min())*1e-9
        angle = (dAmp/config.tel.telDiam) * (180./numpy.pi) * 3600

    A = numpy.vstack([tilts_asec, numpy.ones(len(tilts_asec))]).T
    grad, bias = numpy.linalg.lstsq(A, measuredTilts)[0]
    measPxlScale = 1./grad
    print("Measured Pixel Scale: {} pxls/arcsec".format(measPxlScale))

    return measPxlScale

In [51]:
pxlScales = numpy.linspace(0.05, 0.5, 10)
measPxlScales = numpy.zeros_like(pxlScales)
for ip, p in enumerate(pxlScales):
    config.wfss[0].subapFOV = config.wfss[0].pxlsPerSubap*p
    measPxlScales[ip] = getMeasurePixelScale(4*p, config, samples=20)


Measured Pixel Scale: 20.3655668852 pxls/arcsec
Measured Pixel Scale: 10.1600365369 pxls/arcsec
Measured Pixel Scale: 6.75993523843 pxls/arcsec
Measured Pixel Scale: 5.06164695175 pxls/arcsec
Measured Pixel Scale: 3.97398068907 pxls/arcsec
Measured Pixel Scale: 3.32796896392 pxls/arcsec
Measured Pixel Scale: 2.86908211355 pxls/arcsec
WFS.py -> initFFTs: requested WFS FFT Padding less than FOV size... Setting oversampling to: 4
Measured Pixel Scale: 2.51618072463 pxls/arcsec
Measured Pixel Scale: 2.23332223378 pxls/arcsec
WFS.py -> initFFTs: requested WFS FFT Padding less than FOV size... Setting oversampling to: 5
Measured Pixel Scale: 2.01509244836 pxls/arcsec


In [55]:
plt = bplot.figure(x_axis_label="Input Pixel Scale", y_axis_label="Measured Pixel Scale")
plt.line(pxlScales, measPxlScales)
plt.line(pxlScales, pxlScales, color="black"
    )
bplot.show(plt)